**Лабораторная работа №5. Выделение контуров на изображении**

In [1]:
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import invert
import matplotlib.pyplot as plt
from scipy.spatial import distance
import pandas as pd
import numpy as np
import math
import sys
import csv
import os

1. Сгенерировать эталонные изображения символов.
Алфавит - Тибетский

In [15]:
def create_image_from_letter(font_path, letter, output_path):
  image = Image.new('RGB', (100, 120), color = (255, 255, 255))
  font = ImageFont.truetype(font_path, size = 90)

  # Создаем объект для рисования текста
  draw = ImageDraw.Draw(image)

  # Рисуем букву в центре изображения
  draw.text((50, 50), letter, font=font, fill=(0, 0, 0), anchor="mm")
  image.save(os.path.join(output_path, f"{letter}.png"))
  image.save(f"{letter}.png")
  img = invert(image)
  img.save(os.path.join(output_path, f"{letter}_invert.png"))
  img.save(f"{letter}_invert.png")

2. Для каждого изображения рассчитать набор признаков. Признаки:

  a) Вес (масса чёрного) каждой четверти изображения символа;

  b) Удельный вес (вес, нормированный к четверти площади);

  c) Координаты центра тяжести;

  d) Нормированные координаты центра тяжести;

  e) Осевые моменты инерции по горизонтали и вертикали;

  f) Нормированные осевые моменты инерции.

  g) Профили X и Y

  
3. Скалярные признаки сохранить в текстовый файл формата *.csv (таблица, где
значения в строках разделены точкой с запятой), профили сохранить в *.png в виде
столбчатой диаграммы с правильной ориентацией и целыми числами в подписях
на осях.

In [10]:
# Находим индекс первого и последнего ненулевого элемента вдоль заданной оси
def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)


def last_nonzero(arr, axis, invalid_val=-1):
    mask = arr != 0
    val = arr.shape[axis] - np.flip(mask, axis=axis).argmax(axis=axis) - 1
    return np.where(mask.any(axis=axis), val, invalid_val)


def get_profiles(img):
    return {
        'x': {
            'y': np.sum(img, axis=0),
            'x': np.arange(start=1, stop=img.shape[1] + 1).astype(int)
        },
        'y': {
            'y': np.arange(start=1, stop=img.shape[0] + 1).astype(int),
            'x': np.sum(img, axis=1)
        }
    }


def save_profiles_as_images(img, letter, type):
    profiles = get_profiles(img)

    for axis, profile_data in profiles.items():
        if type == 'x':
            plt.bar(x=profiles['x']['x'], height=profiles['x']['y'], color='blue')
            plt.title(f'Профиль буквы по оси {axis.upper()}')

            # Подпись оси X и оси Y
            plt.xlabel('Сумма значений')
            plt.ylabel('Пиксели')
            plt.savefig(f'profiles_x/profile_{letter}_axis_x.png')
            plt.savefig(f'/profile_{letter}_axis_x.png')
            plt.close()

        elif type == 'y':
            plt.barh(y=profiles['y']['y'], width=profiles['y']['x'], color='blue')
            plt.title(f'Профиль буквы по оси {axis.upper()}')

            # Подпись оси X и оси Y
            plt.xlabel('Сумма значений')
            plt.ylabel('Пиксели')
            plt.savefig(f'profiles_y/profile_{letter}_axis_y.png')
            plt.savefig(f'profile_{letter}_axis_y.png')
            plt.close()

        else:
            print("Некорректное значение type. Пожалуйста, укажите 'x' или 'y'.")
            return
def get_weight(img_bin):
    return np.sum(img_bin)

def get_norm_weight(img_bin):
    return np.sum(img_bin) / img_bin.shape[0] / img_bin.shape[1]

def get_center(img_bin):
    weight = get_weight(img_bin)

    sum_x = 0
    sum_y = 0
    for x in range(img_bin.shape[0]):
        for y in range(img_bin.shape[1]):
            sum_x += x * img_bin[x, y]
            sum_y += y * img_bin[x, y]

    return sum_x / weight, sum_y / weight

def get_relative_center(img_bin):
    x, y = get_center(img_bin)
    return x / img_bin.shape[0], y/img_bin.shape[1]

def get_inertia_moment(img_bin):
    x_c, y_c = get_center(img_bin)
    I_x, I_y = 0, 0

    for x in range(img_bin.shape[0]):
        for y in range(img_bin.shape[1]):
            I_x += (x - x_c)**2 * img_bin[x, y]
            I_y += (y - y_c)**2 * img_bin[x, y]

    return I_x, I_y

def get_norm_inertia_moment(img_bin):
    I_x, I_y = get_inertia_moment(img_bin)
    norm = (img_bin.shape[0] * img_bin.shape[1])**2
    return I_x / norm, I_y / norm


In [17]:
def binarized(img):
    return np.array(img) > 255//2

def save_characteristics_to_csv():
    alphabet = list('ཀཁགངཅཆཇཉཏཐདནཔཕབམཙཚཛཝཞཟའཡརལཤསཧཨཀྵཪཫཬ')

    weight = []
    norm_weight = []
    center = []
    rel_center = []
    inertia = []
    norm_intertia = []

    directory = os.fsencode('alphabet')
    indexes = []


    for i in range(0, 30):
        filename = f"{alphabet[i]}.png"
        img = Image.open(filename).convert('L')
        img_bin = binarized(np.array(img))
        weight.append(get_weight(img_bin))
        norm_weight.append(get_norm_weight(img_bin))
        center.append(get_center(img_bin))
        rel_center.append(get_relative_center(img_bin))
        inertia.append(get_inertia_moment(img_bin))
        norm_intertia.append(get_norm_inertia_moment(img_bin))

    data = {
        'weight': weight,
        'norm_weight': norm_weight,
        'center': center,
        'rel_center': rel_center,
        'inertia': inertia,
        'norm_inertia': norm_intertia
    }


    alphabet = alphabet[:30]  # Обрезаем
    df = pd.DataFrame(data, index=alphabet)
    df.to_csv('/content/characteristics.csv', index_label='letter')


def main():
    font_path = "/content/himalaya.ttf"
    TIBET_LETTERS = [
        'ཀ', 'ཁ', 'ག', 'ང', 'ཅ', 'ཆ', 'ཇ', 'ཉ',
        'ཏ','ཐ', 'ད', 'ན', 'པ', 'ཕ', 'བ', 'མ',
        'ཙ', 'ཚ','ཛ', 'ཝ', 'ཞ', 'ཟ', 'འ', 'ཡ',
        'ར', 'ལ', 'ཤ','ས', 'ཧ', 'ཨ', 'ཀྵ', 'ཪ',
        'ཫ', 'ཬ'
    ]
    output_dir = "/content/output/"
    characteristics_list = []  # Создаем пустой список для хранения характеристик

    for letter in TIBET_LETTERS:
      output_path = output_dir
      create_image_from_letter(font_path, letter, output_path)

    for letter in TIBET_LETTERS:
      # Получаем характеристики для каждого изображения
      image_path = os.path.join(output_dir, f'{letter}.png')
      image = Image.open(image_path)


      output_path = os.path.join(output_dir, f"{letter}.png")

      image_path = os.path.join(output_dir, f'{letter}.png')
      image = Image.open(image_path).convert('L')
      img_src_arr = np.array(image)

      img_src_arr[img_src_arr == 0] = 1
      img_src_arr[img_src_arr == 255] = 0

      save_profiles_as_images(img_src_arr, letter, type='x')
      save_profiles_as_images(img_src_arr, letter, type='y')

      weight = []
      norm_weight = []
      center = []
      rel_center = []
      inertia = []
      norm_intertia = []

      # Сохраняем характеристики в файл CSV
      save_characteristics_to_csv()




if __name__ == '__main__':
    main()